In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt

In [3]:
def dateparse(time):
    date = datetime.datetime.fromtimestamp(float(time))
    return date


data = pd.read_csv(r'C:\Users\Henry\DS\NPD6.csv', index_col=[0])


## NPD Data
Top selling games data from __[the NPD group website](https://www.interactive.org/npd/index.asp)__. Only includes rankings, no sales figures.

In [4]:
data

,October 2018 NPD Games Sales,September 2018 NPD Games Sales,August 2018 NPD Games Sales,July 2018 NPD Games Sales,May 2018 NPD Games Sales,April 2018 NPD Games Sales,March 2018 NPD Games Sales,February 2018 NPD Games Sales,December 2017 NPD Games Sales,October 2017 NPD Games Sales,...,August 2008 NPD Games Sales,July 2008 NPD Games Sales,May 2008 NPD Games Sales,April 2008 NPD Games Sales,February 2008 NPD Games Sales,January 2008 NPD Games Sales,December 2007 NPD Games Sales,September 2007 NPD Games Sales,August 2007 NPD Games Sales,June 2007 NPD Games Sales
month,,,,,,,,,,,,,,,,,,,,,
NaN,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,...,Top 10 SKUs for July 2008,Overall Top 20 Video Game Titles,Overall Top 20 Video Game Titles,Top 20 Video Game Titles,Overall Top 20 Video Game Titles,Overall Top 10 Video Games,Overall Top 20 Video Games,Overall Top 20 Video Games,Overall Top 20 Video Game Titles,Overall Top 20 Video Game Titles
1.0,Call of Duty: Black Ops 4,Spider-Man,Madden NFL 19^,Octopath Traveler*,State of Decay 2,God of War,Far Cry 5,Monster Hunter: World,Call of Duty: WWII,Middle Earth: Shadow of War,...,NCAA Football 09 (360),Metal Gear Solid 4: Guns of the Patriots (PS3),Grand Theft Auto IV* (Xbox 360),Grand Theft Auto IV* (Xbox 360),Call of Duty 4: Modern Warfare* (360),Call of Duty 4: Modern Warfare (Xbox 360),Call of Duty 4: Modern Warfare* (Xbox 360),Halo 3 (Xbox 360),Madden NFL 08 (Xbox 360),Mario Party 8 (Nintendo Wii)
2.0,Red Dead Redemption 2,NBA 2K19,Monster Hunter: World,Grand Theft Auto V,God of War,Far Cry 5,Sea Of Thieves,Call of Duty: WWII,Star Wars: Battlefront II ^,Assassin's Creed: Origins,...,Wii Fit (Wii),Guitar Hero: On Tour (NDS),Mario Kart Wii,Mario Kart Wii (Wii),Devil May Cry 4* (360),Wii Play (Wii),Super Mario Galaxy (Wii),Wii Play (Wii),Madden NFL 08 (Playstation 2),Wii Play (Nintendo Wii)
3.0,Assassin's Creed: Odyssey,Assassin's Creed: Odyssey,Tom Clancy's Rainbow Six: Siege,Mario Kart 8*,Detroit: Become Human,MLB 18: The Show,MLB 18: The Show,NBA 2K18,Super Mario Odyssey**,Super Mario Odyssey**,...,Guitar Hero: On Tour (NDS),Ninja Gaiden II (Xbox 360),Wii Fit (wii),Grand Theft Auto IV* (PS3),Wii Play (Wii),Guitar Hero III: Legends of Rock (Wii),Guitar Hero III: Legends of Rock* (PS2),The Legend of Zelda: Phantom Hourglass (NDS),Bioshock (Xbox 360),Pokemon Diamond Version (Nintendo DS)
4.0,NBA 2K19,FIFA 19^,Grand Theft Auto V,Crash Bandicoot: N. Sane Trilogy,Far Cry 5,Labo Variety Kit,Kirby Star Allies*,Dragon Ball: Fighterz,NBA 2K18,South Park: The Fractured But Whole,...,Wii Play (Wii),Wii Fit (Wii),Grand Theft Auto IV* (PS3),Wii Play (Wii),Devil May Cry 4* (PS3),Rock Band Special Edition Bundle (Xbox 360),Wii Play (Wii),Madden NFL 08 (Playstation 2),Madden NFL 08 (Playstation 3),Pokemon Pearl Version (Nintendo DS)
5.0,Super Mario Party*,Shadow Of The Tomb Raider,Mario Kart 8*,The Legend of Zelda: Breath of the Wild*,Donkey Kong Country: Tropical Freeze*,Donkey Kong Country: Tropical Freeze*,Grand Theft Auto V,Grand Theft Auto V,Mario Kart 8**,NBA 2K18,...,NCAA Football 09 (PS3),Wii Play (Wii),Wii Play (Wii),Super Smash Bros. Brawl (Wii),Guitar Hero III: Legends of Rock* (Wii),Guitar Hero III: Legends of Rock (Xbox 360),Assassin's Creed* (Xbox 360),Skate (Xbox 360),Wii Play (Wii),Forza Motorsport 2 (Xbox 360)
6.0,Soul Calibur VI,Madden NFL 19^,Naruto To Boruto: Shinobi Striker,Far Cry 5,Grand Theft Auto V,Mario Kart 8*,Call of Duty: WWII,Shadow of Colossus 2018,Madden NFL 18,FIFA 18,...,Soul Calibur IV* (360),Battlefield: Bad Company* (Xbox 360),Super Smash Bros. Brawl (Wii),Gran Turismo 5: Prologue (PS3),Mario and Sonic at the Olympic Games (NDS),Super Mario Galaxy (Wii),Halo 3* (Xbox 360),Madden NFL 08 (Xbox 360),Metroid Prime 3: Corruption (Wii),Guitar Hero II w/ guitar (Playstation 2)
7.0,FIFA 19^,Forza Horizon 4,God of War,Super Mario Odyssey*,Hyrule Warriors*,Grand Theft Auto V,Mario Kart 8*,UFC 3,P

Some months only list top 10 instead of top 20, hence the NaN entries.

Also, the names need to be cleaned up due to *'s and ^'s listed on the site.

I will make use of Google trends' suggested title function to get better search results when a good match is available.

In [5]:
monthlist = list(data.columns.values)

from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

def cleanname(game): #extract just the name of the game
    if '(' in game:
        a = game.find('(')
        name = game[:a-1]
    else:
        name = game
    while '*' in name:
        b = name.find('*')
        name = name[:b] + name[b+1:]
    if '^' in name:
        b = name.find('^')
        name = name[:b] + name[b+1:]
    return name

from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def bestmatch(game): #try and get a match for the game, gives better search results
    match = game
    match_n = game
    try:
        googsuggest = pytrends.suggestions(game)
    except:
        googsuggest = []
    s = 0
    for i in range(0,len(googsuggest)):
        name = googsuggest[i]['title']
        s_type = googsuggest[i]['type']
        mid = googsuggest[i]['mid']
        sim = similar(name.lower(),game.lower()) #only looking for video title game matches. Want to avoid the entire series ex: Mario bros series
        if sim > s and ('Video game' in s_type or 'video game' in s_type or 'Console game' in s_type or 'Computer game' in s_type or 'Online game' in s_type or 'Musical game' in s_type or 'Survival game' in s_type):
            s = sim
            match = mid
            match_n = name
    if s < 0.925: #forced to be high similarity to avoid similar titles but different games
        match = game 
    return [match,match_n]
        

In [6]:
fullgamelist= []
for i in range(0,len(monthlist)):
    for j in range(1,21):
        game = data.iloc[j][monthlist[i]]
        if type(game) == str:
            name = cleanname(game)
            data.iloc[j][monthlist[i]] = name
            fullgamelist.append(name)
fullgamelist = set(fullgamelist)

In [7]:
import pickle
with open('data.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)
    
with open('fullgamelist.pickle','wb') as handle:
    pickle.dump(fullgamelist,handle,protocol=pickle.HIGHEST_PROTOCOL)
    

In [8]:
fullgames = [game for game in fullgamelist]
gamematches = []
for i in range(0,len(fullgames)):
    [match,match_n] = bestmatch(fullgames[i])
    gamematches.append([fullgames[i],match_n,match]) #[game name, google suggested name, googletrends match code]

In [9]:
data #now with names cleaned up

,October 2018 NPD Games Sales,September 2018 NPD Games Sales,August 2018 NPD Games Sales,July 2018 NPD Games Sales,May 2018 NPD Games Sales,April 2018 NPD Games Sales,March 2018 NPD Games Sales,February 2018 NPD Games Sales,December 2017 NPD Games Sales,October 2017 NPD Games Sales,...,August 2008 NPD Games Sales,July 2008 NPD Games Sales,May 2008 NPD Games Sales,April 2008 NPD Games Sales,February 2008 NPD Games Sales,January 2008 NPD Games Sales,December 2007 NPD Games Sales,September 2007 NPD Games Sales,August 2007 NPD Games Sales,June 2007 NPD Games Sales
month,,,,,,,,,,,,,,,,,,,,,
NaN,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,Top 20 Games,...,Top 10 SKUs for July 2008,Overall Top 20 Video Game Titles,Overall Top 20 Video Game Titles,Top 20 Video Game Titles,Overall Top 20 Video Game Titles,Overall Top 10 Video Games,Overall Top 20 Video Games,Overall Top 20 Video Games,Overall Top 20 Video Game Titles,Overall Top 20 Video Game Titles
1.0,Call of Duty: Black Ops 4,Spider-Man,Madden NFL 19,Octopath Traveler,State of Decay 2,God of War,Far Cry 5,Monster Hunter: World,Call of Duty: WWII,Middle Earth: Shadow of War,...,NCAA Football 09,Metal Gear Solid 4: Guns of the Patriots,Grand Theft Auto IV,Grand Theft Auto IV,Call of Duty 4: Modern Warfare,Call of Duty 4: Modern Warfare,Call of Duty 4: Modern Warfare,Halo 3,Madden NFL 08,Mario Party 8
2.0,Red Dead Redemption 2,NBA 2K19,Monster Hunter: World,Grand Theft Auto V,God of War,Far Cry 5,Sea Of Thieves,Call of Duty: WWII,Star Wars: Battlefront II,Assassin's Creed: Origins,...,Wii Fit,Guitar Hero: On Tour,Mario Kart Wii,Mario Kart Wii,Devil May Cry 4,Wii Play,Super Mario Galaxy,Wii Play,Madden NFL 08,Wii Play
3.0,Assassin's Creed: Odyssey,Assassin's Creed: Odyssey,Tom Clancy's Rainbow Six: Siege,Mario Kart 8,Detroit: Become Human,MLB 18: The Show,MLB 18: The Show,NBA 2K18,Super Mario Odyssey,Super Mario Odyssey,...,Guitar Hero: On Tour,Ninja Gaiden II,Wii Fit,Grand Theft Auto IV,Wii Play,Guitar Hero III: Legends of Rock,Guitar Hero III: Legends of Rock,The Legend of Zelda: Phantom Hourglass,Bioshock,Pokemon Diamond Version
4.0,NBA 2K19,FIFA 19,Grand Theft Auto V,Crash Bandicoot: N. Sane Trilogy,Far Cry 5,Labo Variety Kit,Kirby Star Allies,Dragon Ball: Fighterz,NBA 2K18,South Park: The Fractured But Whole,...,Wii Play,Wii Fit,Grand Theft Auto IV,Wii Play,Devil May Cry 4,Rock Band Special Edition Bundle,Wii Play,Madden NFL 08,Madden NFL 08,Pokemon Pearl Version
5.0,Super Mario Party,Shadow Of The Tomb Raider,Mario Kart 8,The Legend of Zelda: Breath of the Wild,Donkey Kong Country: Tropical Freeze,Donkey Kong Country: Tropical Freeze,Grand Theft Auto V,Grand Theft Auto V,Mario Kart 8,NBA 2K18,...,NCAA Football 09,Wii Play,Wii Play,Super Smash Bros. Brawl,Guitar Hero III: Legends of Rock,Guitar Hero III: Legends of Rock,Assassin's Creed,Skate,Wii Play,Forza Motorsport 2
6.0,Soul Calibur VI,Madden NFL 19,Naruto To Boruto: Shinobi Striker,Far Cry 5,Grand Theft Auto V,Mario Kart 8,Call of Duty: WWII,Shadow of Colossus 2018,Madden NFL 18,FIFA 18,...,Soul Calibur IV,Battlefield: Bad Company,Super Smash Bros. Brawl,Gran Turismo 5: Prologue,Mario and Sonic at the Olympic Games,Super Mario Galaxy,Halo 3,Madden NFL 08,Metroid Prime 3: Corruption,Guitar Hero II w/ guitar
7.0,FIFA 19,Forza Horizon 4,God of War,Super Mario Odyssey,Hyrule Warriors,Grand Theft Auto V,Mario Kart 8,UFC 3,Playerunknown's Battlegrounds,WWE 2K18,...,Mario Kart Wii,Mario Kart Wii,Iron Man,Pokemon Mystery Dungeon: Explorers of Darkness,Lost Odyssey,Burnout Paradise,Brain Age 2: More Training in Minutes a Day,Metroid Prime 3: Corruption,Mario Strikers: Charged,Guitar Hero II w/ guitar
8.0,Spider-Man,Destiny 2,Call Of Duty: Modern Warfare 2,The Crew 2,Tom Clancy's Rainbow Six: Siege,Call of Duty: WWII,Ni No Kuni II: Revenant Kingdom,Kingdom Come: Deliverance,Assassin's Creed: Origins,Madden NFL 18,...,Rock Band,Lego Indiana Jones: The Original Adve

Now to add more games taken from __[gamerankings](https://www.gamerankings.com)__, to include trend data for games that never chart.

In [10]:
gamesreleased = pd.read_csv(r'C:\Users\Henry\DS\gamesreleased.csv', index_col=[0])

extragamelist= []
for i in range(0,gamesreleased.shape[0]):
    if i % 2 == 0:
        game = gamesreleased.index.values[i]
        try:
            name = cleanname(game)
            extragamelist.append(name)
        except:
            print("skip")
        
extragamematches1 = []
for i in range(0,len(extragamelist)):
    [match,match_n] = bestmatch(extragamelist[i])
    extragamematches1.append([extragamelist[i],match_n,match])

In [11]:
extragamematches = [element for element in extragamematches1 if element not in gamematches]

In [13]:
with open('gamematches.pickle','wb') as handle:
    pickle.dump(gamematches,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
with open('NPDdata.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)